<a href="https://colab.research.google.com/github/sudharshanpaul/Deep-Learning/blob/main/Boltzmann_Machines/MovieRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Boltzmann Machines

#### Importing the Libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

### Data Preprocessing

#### Importing the Libraries

In [ ]:
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine='python', encoding='latin-1')
users = pd.read_csv('users.dat', sep = '::', header = None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine='python', encoding='latin-1')

In [ ]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


#### Preparing the Training and Testing set

In [ ]:
training_set = pd.read_csv('u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')

test_set = pd.read_csv('u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

In [ ]:
print(f'training_set shape: {training_set.shape}')
print(f'test_set shape: {test_set.shape}')

training_set shape: (79999, 4)
test_set shape: (19999, 4)


#### Getting the number of users and movies

In [ ]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:, 1])))

In [ ]:
print(f'Number of users: {nb_users}')
print(f'Number of movies: {nb_movies}')

Number of users: 943
Number of movies: 1682


#### Converting the data into an array with users in the rows and movies in the columns

In [ ]:
def convert(data):
  new_data = []
  for id_user in range(1, nb_users + 1):
    id_movies = data[:, 1][data[:, 0] == id_user]
    id_ratings = data[:, 2][data[:, 0] == id_user]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))

  return new_data

In [ ]:
training_set = convert(training_set)
test_set = convert(test_set)

In [ ]:
training_set[0][:5]

[np.float64(0.0),
 np.float64(3.0),
 np.float64(4.0),
 np.float64(3.0),
 np.float64(3.0)]

#### Converting the data into *Torch Tensors*

In [ ]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

#### Converting the ratings into *binary ratings* where 1 ( liked ) and 0 ( unliked )

In [ ]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

In [ ]:
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

### Creating the architecture of Neural Network

In [ ]:
class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nv, nh)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)

  ## Sampling the hidden nodes - here x corresponds to visible neurons
  def sample_h(self, x):
    wx = torch.mm(x, self.W)  # x: [batch_size, nv], W: [nv, nh] => wx: [batch_size, nh]
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)

  def sample_v(self, y):
    wy = torch.mm(y, self.W.t())  # y: [batch_size, nh], W.t(): [nh, nv] => wy: [batch_size, nv]
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)


  def train(self, v0, vk, ph0, phk):
    self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)



In [ ]:
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

### Training the RBM

In [ ]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, nb_users - batch_size, batch_size):
    vk = training_set[id_user:id_user+batch_size]
    v0 = training_set[id_user:id_user+batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
    s += 1.
  print(f'epoch: {epoch} loss: {train_loss/s}')

epoch: 1 loss: 0.24137255549430847
epoch: 2 loss: 0.24829711019992828
epoch: 3 loss: 0.24684445559978485
epoch: 4 loss: 0.24931424856185913
epoch: 5 loss: 0.24498918652534485
epoch: 6 loss: 0.24783611297607422
epoch: 7 loss: 0.24799799919128418
epoch: 8 loss: 0.24487532675266266
epoch: 9 loss: 0.2461007982492447
epoch: 10 loss: 0.24742142856121063


### Testing the RBM

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])) # Average Distance here
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2452)


- We got nearly 75% accuracy